In [1]:
!pip install transformers huggingface_hub torch

# Imports

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
import huggingface_hub

# Constants

In [3]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
NLI_MODEL_NAME = "MoritzLaurer/mDeBERTa-v3-base-xnli-multilingual-nli-2mil7"
MODEL_NAME = "meta-llama/Llama-3.2-1B"

# Perform QA

In [4]:
huggingface_hub.login()

In [7]:
premise = "tell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News"
model_inference = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
tokenizer_inference = AutoTokenizer.from_pretrained(MODEL_NAME)

input = tokenizer_inference(premise, truncation=True, return_tensors="pt")
hypothesis = model_inference.generate(input["input_ids"].to(device), max_length=500)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [11]:
out = [tokenizer_inference.decode(id) for id in hypothesis]
out

["<|begin_of_text|>tell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and Gadd - Morgan - BBC News\ntell me about the news without searching any sources: Baby Reindeer 'a big problem' for Netflix and

# Perform evaluation with NLI

In [ ]:
tokenizer_NLI = AutoTokenizer.from_pretrained(NLI_MODEL_NAME)
model_NLI = AutoModelForSequenceClassification.from_pretrained(NLI_MODEL_NAME)

input = tokenizer_NLI(premise, hypothesis, truncation=True, return_tensors="pt")
output = model_NLI(input["input_ids"].to(device))
prediction = torch.softmax(output["logits"][0], -1).tolist()
label_names = ["entailment", "neutral", "contradiction"]
prediction = {name: round(float(pred) * 100, 1) for pred, name in zip(prediction, label_names)}
print(prediction)

c:\Users\wsr20\anaconda3\envs\CoT\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


{'entailment': 84.1, 'neutral': 9.1, 'contradiction': 6.8}


# Perform evaluation with LLaMa

In [ ]:
eval_prompt = "Is this response either responding to content irrelevant to the question, making conflicts to the content in the response, or making use of incorrect assumptions? Answer with yes or no without explanation: "
eval_result = model_inference.generate(eval_prompt)[0]